In [1]:
import os
import json
import pandas as pd
import numpy as np
import tqdm
import scipy.sparse as sp

import implicit
import lightfm
import warnings
from validation_prepare import *
import matplotlib.pyplot as plt
import seaborn as sns
warnings.filterwarnings('ignore')

C:\Users\yabom\Anaconda3\lib\site-packages\lightfm\_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn('LightFM was compiled without OpenMP support. '


In [2]:
pd.set_option('display.max_columns',100)

DATA_PATH = '../okko/orig_data'
PREPARED_PATH = './prepared_data/'

In [3]:
actions = pd.read_pickle(PREPARED_PATH+'actions_one_table.pkl')

actions.sort_index(inplace = True) # На всякий случай, иначе деление не будет работать

actions.head()

action consumption_mode  \
user_uid element_uid ts                                     
0        51          4.416546e+07  watch                S   
         72          4.375829e+07  watch                S   
         207         4.371904e+07  watch                S   
         209         4.377814e+07  watch                S   
         434         4.338109e+07  watch                S   

                                   device_manufacturer  device_type  rating  \
user_uid element_uid ts                                                       
0        51          4.416546e+07                 99.0          0.0     NaN   
         72          4.375829e+07                 99.0          0.0     NaN   
         207         4.371904e+07                 99.0          0.0     NaN   
         209         4.377814e+07                 99.0          0.0     NaN   
         434         4.338109e+07                 99.0          0.0     NaN   

                                   watched_time  duration  type  
user_uid element_uid ts                                          
0        51          4.416546e+07       12382.0      3600     1  
         72          4.375829e+07        5653.0      6000     1  
         207         4.371904e+07        2646.0      5400     1  
         209         4.377814e+07        6971.0      7200     1  
         434         4.338109e+07        5894.0      6600     1

In [4]:
idx = get_train_test(actions)

6558458
2186152
2186153


In [ ]:
actions.iloc[idx[0]].index.get_level_values(2).max()

43362401.96226887

In [ ]:
actions.iloc[idx[1]].index.get_level_values(2).min()

43362401.97085199

In [ ]:
actions.iloc[idx[1]].index.get_level_values(2).max()

43828341.47903843

In [ ]:
actions.iloc[idx[2]].index.get_level_values(2).min()

43828341.48519237

In [ ]:
actions.consumption_mode.value_counts()

S    8296227
P     873834
R     472951
Name: consumption_mode, dtype: int64

In [ ]:
#actions.sort_index(level = [0,2,1],inplace = True)

In [ ]:
# Вроде не пересекается.
train,test,valid = actions.iloc[idx[0]],actions.iloc[idx[1]],actions.iloc[idx[2]]

In [ ]:
%%time 
dur_being_train,dur_films_train,watch_actions_train,target_train = get_target(train)
dur_being_test,dur_films_test,watch_actions_test,target_test = get_target(test)
dur_being_valid,dur_films_valid,watch_actions_valid,target_valid = get_target(valid)

Wall time: 42.5 s


In [ ]:
# Получили фичи для фильмов
import pickle
with open(PREPARED_PATH+'catalogue_features.pkl','rb') as f:
    match_element_row,match_row_element,match_columns,element_matrix = pickle.load(f)
movie_match_columns = {i:ii for ii,i in enumerate(match_columns)}
movie_columns_match = {ii:i for ii,i in enumerate(match_columns)}

with open(PREPARED_PATH+'bag_of_attr_movie.pkl','rb') as f:
    bag_of_attr = pickle.load(f)

In [ ]:
'''
Здесь начинается русское поле экспериментов над параметрами моделей.
'''

'\nЗдесь начинается русское поле экспериментов над параметрами моделей.\n'

In [ ]:
_,_,watch_actions,_ = get_target(actions)

actions = actions.join(watch_actions['rel_dur'])
actions['rel_dur'] = actions['rel_dur'].fillna(0).replace(np.inf,1)

In [ ]:
actions['action_enc'] = actions['action'].map({'watch':0,'rate':1,'bookmate':3})
actions['cons_enc'] = actions['consumption_mode'].map({'S':0,'P':1,'R':3})

In [ ]:
actions.sort_index(level = [0,2,1],inplace = True)

In [ ]:
#actions.reset_index()[['user_uid','element_uid']]

In [ ]:
%%time
wm = actions.groupby(level = [0,1],sort = False)[['rel_dur','type','rating','action_enc','cons_enc']].mean()#.agg({'rel_dur':np.mean,'type':np.mean,'rating':np.mean,'action_enc':np.mean,})

In [ ]:
wm.head()

In [ ]:
aa = wm.action_enc.value_counts().to_frame()
aa.columns = ['counts',]
aa['mapa'] = ['watch','bookmate','watch+rate','watch+bookmate','rate','watch+rate+bookmate','rate+bookmate']

In [ ]:
wm.cons_enc.value_counts().to_frame()

In [ ]:
wm['action_enc'] = wm['action_enc'].map(aa['mapa'])
wm['cons_enc'] = wm['cons_enc'].map({0:'S',1:'P',3:'R'})

In [ ]:
# Щас еще приджойним тип потребления фильма и прям ок
# И по хорошему надо бы расположить их по времени, когда их там начали потреблять
ts_df = actions.reset_index().drop_duplicates(subset = ['user_uid','element_uid'])[ ['user_uid','element_uid','ts']]

In [ ]:
ts_df.set_index( ['user_uid','element_uid'],inplace = True)
ts_df.columns = ['first_ts']

In [ ]:
wm = wm.join(ts_df)

In [ ]:
wm.head()

In [ ]:
#wm.to_pickle(PREPARED_PATH + 'u_m_feat.pkl')

In [ ]:
wm['first_ts_1'] = wm['first_ts'].shift(1)

In [ ]:
wm['diff_ts'] = wm['first_ts'] - wm['first_ts_1']

In [ ]:
first_movie = wm.reset_index().groupby('user_uid')['element_uid'].agg(lambda x:x.iloc[0]).to_frame()
first_movie.columns = ['element_uid',]
first_movie.index.name = 'user_uid'
first_movie['ones'] = 1
first_movie.reset_index(inplace = True)
first_movie.set_index(['user_uid','element_uid'],inplace = True)


In [ ]:
first_movie.head()

In [ ]:
#first_movie.index.values

In [ ]:
ind_nan = wm.index.isin(first_movie.index.values)

In [ ]:
#wm.diff_ts.mask(ind_nan, np.nan)
wm.ix[ind_nan,'diff_ts'] = np.nan

In [ ]:
sum(ind_nan)

In [ ]:
#wm.head(100)

In [ ]:
data_catalogue = pd.read_pickle(PREPARED_PATH+'data_catalogue.pkl')

In [ ]:
sns.distplot(data_catalogue.feature_1)

In [ ]:
sns.distplot(data_catalogue.feature_2)

In [ ]:
data_catalogue.feature_2.max()

In [ ]:
sns.countplot(data_catalogue.feature_3)

In [ ]:
sns.distplot([*data_catalogue.feature_4,*data_catalogue.feature_2])

In [ ]:
data_catalogue.feature_4.max()

In [ ]:
sns.countplot(data_catalogue.feature_5)

In [ ]:
sum(data_catalogue.feature_5.value_counts().index)

In [ ]:
#sns.distplot(wm.diff_ts.dropna())

In [ ]:
# Из чатика мы узнали, что вероятнее всего 184 дня приходится на трейн
#(actions.index.get_level_values(2).max() - actions.index.get_level_values(2).min())/184//1000
nearly_day = 14000

In [ ]:
data_catalogue.index.name = 'element_uid'

In [ ]:
wm = wm.join(data_catalogue[['duration','feature_1','feature_2','feature_3','feature_4','feature_5',]])#.loc[0,'diff_ts'].hist()

In [ ]:
begin_ts = actions.index.get_level_values(2).min()
end_ts = actions.index.get_level_values(2).max()

In [ ]:
#wm['feature_2'].hist()
wm[wm.first_ts < wm.feature_1]
# feature_1 - это видимо время премьеры.
# Значит можем ориентироваться, насколько есть старые фильмым
# Давайте тогда гденить в топе рекомендовать то, что добавлено в избранное и является новинкоц
# а то, что новинка сунем с понижающим коэффициентом, наверное...хотя
wm['is_history'] = 1*(wm['feature_1']<begin_ts)
wm['is_future'] = 1*(wm['feature_1']> end_ts)



In [ ]:
wm['is_history'].value_counts()

In [ ]:
wm['is_future'].value_counts()

In [ ]:
#Странно, потому что вроде как у них просто получается не дошли руки до свроего же избранного
# Или типа там именно избранное, а не жду с нетерпением


In [ ]:
wm['diff_novice_ts'] = (wm['feature_1'] - wm['first_ts'])//14000//365

In [ ]:
wm.diff_novice_ts.hist()

In [ ]:
# Это не дата премьеры, это дата, когда кино появилось на платформе, ну или там стало доступно для просмотра.год основания okko 2011


In [ ]:
wm.head(40)

In [ ]:

_, bins = np.histogram(wm['rel_dur'])
# g = sns.FacetGrid(wm, hue="feature_5",)
# g = g.map(sns.distplot, "rel_dur", bins=bins,ax = ax)
for i in np.unique(wm["feature_5"].values):
    fig,ax = plt.subplots(figsize = (10,10))
    sns.distplot(wm.loc[wm.feature_5 == i,'rel_dur'],bins = bins)

In [ ]:
# Надо как-то формалт=изовать зашло не зашло.
# Например, ситуация - человек сел, посмотрел мало (т.е. не зашло, а значит перебирает сколько то фильмов пока не найдет тот, который зайдет)
# , захотел смотреть больше и подряд.
# Можно вычислить сначала такие последовательности, а потом критическое время просмотра в минутах.


In [ ]:
'''
Здесь заканчиваетс/ русское поле экспериментов
'''

In [ ]:
actions.head()

In [ ]:
#Проверим, что не загибается на полном датасете

fe = FeatureExtractor({'movie_attr_matrix':element_matrix,'movie_match_columns':movie_match_columns,
                       'movie_columns_match':movie_columns_match,'movie_match_row_movie':match_row_element,
                      'movie_match_movie_row':match_element_row,},bag_of_attr,mode = 'duration',delimiter = 1/3,
                     target_col_name = 'rel_dur',)


# fe = FeatureExtractor({'movie_attr_matrix':element_matrix,'movie_match_columns':movie_match_columns,
#                        'movie_columns_match':movie_columns_match,'movie_match_row_movie':match_row_element,
#                       'movie_match_movie_row':match_element_row,},bag_of_attr,is_censor = False)
X = fe.fit_transform(actions)

In [ ]:
cfe = ColdFeatureExtractor(fe)

res_action = cfe.fit_transform(actions)

In [ ]:
res_action.keys()

In [ ]:
len(fe.train_movie_match_movie_row),len(fe.match_user_row)

In [ ]:
with open(os.path.join(DATA_PATH, 'test_users.json'), 'r') as f:
    test_users = set(json.load(f)['users'])

In [ ]:
X_user = res_action['train_user']
X_movie = res_action['train_movie']
X_user.shape,X_movie.shape

In [ ]:
# def fit_lightfm(train,item_features=None,seed = 0)
seed = 0
epochs = 30
num_threads=4

model = lightfm.LightFM(loss = 'warp',random_state=seed)


# model.fit(train_matrix,user_features = train_user,item_features = element_matrix,epochs = epochs,num_threads = num_threads,
#          verbose = True)


In [ ]:


%time model.fit(X,user_features=X_user,item_features=X_movie,epochs = epochs,num_threads = num_threads,verbose = True)

In [ ]:
%%time
a,b = X.nonzero()
D = pd.DataFrame([a,b],index = ['row','columns']).T
D = D.groupby('row').agg(lambda x:frozenset(x.values))
D

In [ ]:
D.loc[1]

In [ ]:
len(fe.train_movie_match_row_movie)

In [ ]:
print('sparsity',X.nnz/(X.shape[0]*X.shape[1]))

In [ ]:
len( set(test_users) - set(fe.match_user_row.keys()))

In [ ]:
from implicit.nearest_neighbours import TFIDFRecommender,CosineRecommender,NearestNeighboursScorer



In [ ]:
# Для implicit  надо фильмы*юзеры матрицу.
# model = CosineRecommender()
# model.fit(X.T.tocsr())

In [ ]:
#ratings_matrix_T.shape

In [ ]:
# Сделаем какой-нибудь сабмит из lightfm
result = {}
mode = 'lightfm'#'implicit'#'lightfm'
cnt = 0
for user_uid in tqdm.tqdm(test_users):
    
    # transform user_uid to model's internal user category
    try:
        user_cat = fe.match_user_row[user_uid]
    except LookupError:
        continue
    if mode == 'lightfm':
        need_cols = np.array(list(set(np.arange(X.shape[1])) - D.get(user_cat, set())))
        recs = model.predict(user_cat,need_cols,user_features=X_user,item_features=X_movie,num_threads=num_threads)
        need_movies = np.argsort(-recs)[:20]
        result[user_uid] = [int(fe.train_movie_match_row_movie[i]) for i in need_movies]
    # переводим в фильмы
    
    # perform inference
    if mode == 'implicit':
        if cnt == 0:
            ratings_matrix_T = X.T.tocsr()
        recs = model.recommend(
            user_cat,
            X.tocsr(),
            N=20,
            filter_already_liked_items=True,
            filter_items=D.get(user_uid, set())
        )
        result[user_uid] = [int(fe.train_movie_match_row_movie[i]) for i, _ in recs]
        # drop scores and transform model's internal elelemnt category to element_uid for every prediction
    # also convert np.uint64 to int so it could be json serialized later
    cnt += 1

In [ ]:
user_cat

In [ ]:
len(fe.train_movie_match_row_movie)

In [ ]:
recs

In [ ]:
#len(need_cols),len(recs),type(recs)

In [ ]:
#plt.hist(recs)

In [ ]:
len(result)

In [ ]:

with open('answer.json', 'w') as f:
    json.dump(result, f)

In [ ]:
te.shape

In [ ]:
from lightfm.evaluation import auc_score,precision_at_k

# Compute and print the AUC score
train_auc = precision_at_k(model, te ,tr, k = 20,num_threads=4).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:
new_old_tr_te_2.keys()

In [ ]:
tr = new_old_tr_te_2['train_interactions']
te = new_old_tr_te_2['test_interactions']
uf = new_old_tr_te_2['user_features_train']
tf = new_old_tr_te_2['user_features_test']
print(tr.shape,te.shape,uf.shape,tf.shape)
# tr[(tr < 3) &  (tr>0)] = -1
# tr[(tr > 3)] = 1
# te[(tr < 3) &  (te>0)] = -1
# te[(tr > 3)] = 1

model.fit(train_matrix,user_features=uf,epochs = epochs,num_threads =num_threads,
         verbose = True)

In [ ]:
from scipy.sparse import csr_matrix
import matplotlib.pyplot as plt


In [ ]:
p = model.predict(0,np.arange(tr.shape[1]),user_features=tf[0,:],num_threads=num_threads)

In [ ]:
plt.hist(p)

In [ ]:
test_user[0,:]

In [ ]:
a = model.predict(0,np.arange(train_matrix.shape[1]),user_features=test_user[0,:],num_threads=num_threads)
len(a)

In [ ]:
from lightfm.evaluation import auc_score,precision_at_k

# Compute and print the AUC score
train_auc = precision_at_k(model, train_matrix,user_features = train_user,item_features = element_matrix ,num_threads=num_threads).mean()
print('Collaborative filtering train AUC: %s' % train_auc)

In [ ]:
model.user_feature_map

In [ ]:
def get_answer(test_users_dict,test):
    pass